# Capstone Project
## IBM Data Science Specialization
### Segmenting and Clustering Parishes in Guayaquil and Quito
#### Created by: Xavier Jácome Piñeiros

Import necessary libraries.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Data acquisition and cleaning

Import data from an open source available in a website. Then transform the imported data into a dataframe.

In [ ]:
guayaquil= pd.DataFrame(pd.read_html('http://www.codigopostalecuador.com/guayaquil-876')[0])
guayaquil.drop(guayaquil.columns[4], axis=1, inplace=True)
guayaquil['Código']=guayaquil['Código'].astype(str)
guayaquil['Código']='0'+guayaquil['Código']
guayaquil.rename(columns={'Código':'PostalCode', 'Lugar':'Site', 'Latitud':'Latitude', 'Longitud':'Longitude'},inplace=True)
guayaquil.head(10)

,PostalCode,Site,Latitude,Longitude
0,090101,Ayacucho,-2.20548,-79.89029
1,090102,Bolivar (Sagrario),-2.19923,-79.88992
2,090103,Carbo (Concepcion),-1.81207,-80.23183
3,090115,Chongon,-2.23591,-80.07755
4,090151,Chongon.,-2.23591,-80.07755
5,090104,Febres Cordero,-2.20340,-79.93424
6,090105,Garcia Moreno,-2.20836,-79.89930
7,090150,Guayaquil,-2.20382,-79.89745
8,090152,Juan Gómez Rendón (Progreso),-2.40000,-80.36667
9,090106,Letamendi,-2.21127,-79.90823


In [ ]:
guayaquil.shape

(25, 4)

In [ ]:
# Dataframe info
guayaquil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
PostalCode    25 non-null object
Site          25 non-null object
Latitude      25 non-null float64
Longitude     25 non-null float64
dtypes: float64(2), object(2)
memory usage: 880.0+ bytes


In [ ]:
guayaquil

,PostalCode,Site,Latitude,Longitude
0,090101,Ayacucho,-2.20548,-79.89029
1,090102,Bolivar (Sagrario),-2.19923,-79.88992
2,090103,Carbo (Concepcion),-1.81207,-80.23183
3,090115,Chongon,-2.23591,-80.07755
4,090151,Chongon.,-2.23591,-80.07755
5,090104,Febres Cordero,-2.20340,-79.93424
6,090105,Garcia Moreno,-2.20836,-79.89930
7,090150,Guayaquil,-2.20382,-79.89745
8,090152,Juan Gómez Rendón (Progreso),-2.40000,-80.36667
9,090106,Letamendi,-2.21127,-79.90823


In [ ]:
for columnas in guayaquil.columns:
    print("----"+columnas+"----")
    print(guayaquil[columnas].value_counts())
    print('\n')


----PostalCode----
090115    1
090114    1
090113    1
090150    1
090111    1
090103    1
090153    1
090157    1
090155    1
090154    1
090107    1
090110    1
090104    1
090151    1
090108    1
090109    1
090112    1
090158    1
090106    1
090101    1
090105    1
090152    1
090102    1
090116    1
090156    1
Name: PostalCode, dtype: int64


----Site----
Pascuales                       2
Olmedo (San Alejo)              1
Ximena                          1
Nueve De Octubre                1
Garcia Moreno                   1
Febres Cordero                  1
Sucre                           1
Roca                            1
Ayacucho                        1
Letamendi                       1
Guayaquil                       1
Urdaneta                        1
Puna                            1
Chongon                         1
Tarqui                          1
Juan Gómez Rendón (Progreso)    1
Chongon.                        1
Tenguel                         1
Morro                  

As I noticed that in guayaquil dataframe were two sites duplicated, I decided to eliminate one.

In [ ]:
guayaquil.drop(index={3,14},inplace=True)

In [ ]:
guayaquil.reset_index(inplace=True)

In [ ]:
guayaquil.drop(columns=guayaquil.columns[0],inplace=True)

Now, let's check if everything is ok with the cleaned dataframe.

In [ ]:
guayaquil.head()

,PostalCode,Site,Latitude,Longitude
0,090101,Ayacucho,-2.20548,-79.89029
1,090102,Bolivar (Sagrario),-2.19923,-79.88992
2,090103,Carbo (Concepcion),-1.81207,-80.23183
3,090151,Chongon.,-2.23591,-80.07755
4,090104,Febres Cordero,-2.20340,-79.93424


In [ ]:
for columnas in guayaquil.columns:
    print("----"+columnas+"----")
    print(guayaquil[columnas].value_counts())
    print('\n')

----PostalCode----
090151    1
090114    1
090113    1
090150    1
090111    1
090103    1
090153    1
090157    1
090155    1
090107    1
090110    1
090104    1
090116    1
090108    1
090109    1
090112    1
090158    1
090106    1
090101    1
090105    1
090152    1
090102    1
090156    1
Name: PostalCode, dtype: int64


----Site----
Olmedo (San Alejo)              1
Ximena                          1
Nueve De Octubre                1
Garcia Moreno                   1
Febres Cordero                  1
Sucre                           1
Roca                            1
Ayacucho                        1
Letamendi                       1
Guayaquil                       1
Urdaneta                        1
Puna                            1
Bolivar (Sagrario)              1
Tarqui                          1
Juan Gómez Rendón (Progreso)    1
Chongon.                        1
Tenguel                         1
Morro                           1
Posorja                         1
Carbo (Concep

Let's do the same analysis for Quito.

In [ ]:
quito= pd.DataFrame(pd.read_html('http://www.codigopostalecuador.com/quito-1896')[0])
quito.drop(quito.columns[4], axis=1, inplace=True)
quito.rename(columns={'Código':'PostalCode', 'Lugar':'Site', 'Latitud':'Latitude', 'Longitud':'Longitude'},inplace=True)
quito.head(10)

,PostalCode,Site,Latitude,Longitude
0,170151,Alangasi,-0.30505,-78.41344
1,170101,Alfaro (Chimbacalle),-0.23333,-78.51667
2,170152,Amaguaña,-0.38084,-78.51544
3,170153,Atahualpa (Chabaspamba),-0.18439,-78.49171
4,170129,Belisario Quevedo,-0.16563,-78.51045
5,170102,Benalcazar,-0.18262,-78.48122
6,170154,Calacali,-0.00114,-78.51355
7,170155,Calderon (Carapungo),-0.09749,-78.42251
8,170120,Carcelen,-0.08971,-78.46992
9,170130,Centro Historico,-0.27874,-78.55484


In [ ]:
quito.shape

(86, 4)

In [ ]:
for columnas in quito.columns:
    print("----"+columnas+"----")
    print(quito[columnas].value_counts())
    print('\n')

----PostalCode----
170111    1
170143    1
170135    1
170136    1
170137    1
170138    1
170139    1
170140    1
170141    1
170142    1
170144    1
170133    1
170145    1
170146    1
170147    1
170148    1
170149    1
170150    1
170151    1
170152    1
170134    1
170132    1
170110    1
170121    1
170113    1
170114    1
170115    1
170116    1
170117    1
170118    1
170119    1
170120    1
170122    1
170131    1
170123    1
170124    1
170125    1
170126    1
170127    1
170128    1
170129    1
170130    1
170153    1
170154    1
170155    1
170186    1
170178    1
170179    1
170180    1
170181    1
170182    1
170183    1
170184    1
170185    1
170101    1
170156    1
170102    1
170103    1
170104    1
170105    1
170106    1
170107    1
170108    1
170109    1
170177    1
170176    1
170175    1
170174    1
170157    1
170158    1
170159    1
170160    1
170161    1
170162    1
170163    1
170164    1
170165    1
170166    1
170167    1
170168    1
170169    1
170170   

In [ ]:
quito[quito['Longitude']==-78.51045]

,PostalCode,Site,Latitude,Longitude
4,170129,Belisario Quevedo,-0.16563,-78.51045
26,170106,El Salvador,-0.16563,-78.51045
27,170125,Eloy Alfaro,-0.16563,-78.51045
38,170139,La Argelia,-0.16563,-78.51045
41,170141,La Ferroviaria,-0.16563,-78.51045
63,170144,Ponceano,-0.16563,-78.51045


In [ ]:
quito.drop(index={4,26,27,38,41,63},inplace=True)

In [ ]:
quito.reset_index(inplace=True)

In [ ]:
quito.drop(columns=quito.columns[0],inplace=True)

In [ ]:
for columnas in quito.columns:
    print("----"+columnas+"----")
    print(quito[columnas].value_counts())
    print('\n')

----PostalCode----
170111    1
170110    1
170135    1
170136    1
170137    1
170138    1
170140    1
170142    1
170143    1
170145    1
170146    1
170147    1
170148    1
170149    1
170150    1
170151    1
170152    1
170153    1
170154    1
170134    1
170133    1
170132    1
170120    1
170113    1
170114    1
170115    1
170116    1
170117    1
170118    1
170119    1
170121    1
170131    1
170122    1
170123    1
170124    1
170126    1
170127    1
170128    1
170130    1
170155    1
170156    1
170157    1
170186    1
170179    1
170180    1
170181    1
170182    1
170183    1
170184    1
170185    1
170101    1
170177    1
170102    1
170103    1
170104    1
170105    1
170107    1
170108    1
170109    1
170178    1
170176    1
170158    1
170166    1
170159    1
170160    1
170161    1
170162    1
170163    1
170164    1
170165    1
170167    1
170175    1
170168    1
170169    1
170170    1
170171    1
170172    1
170173    1
170174    1
170112    1
Name: PostalCode, dty

Import libraries that allows to create maps.

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let's create the Guayaquil's map.

In [ ]:
address = 'Guayaquil, EC'

geolocator = Nominatim(user_agent="Guayaquil")
location = geolocator.geocode(address)
latitude_guayaquil = location.latitude
longitude_guayaquil = location.longitude
print('The geograpical coordinate of Guayaquil are {}, {}.'.format(latitude_guayaquil, longitude_guayaquil))

The geograpical coordinate of Guayaquil are -2.1898771, -79.8877153.


In [ ]:
map_guayaquil = folium.Map(location=[latitude_guayaquil, longitude_guayaquil], zoom_start=10)

for lat, lng, Site in zip(guayaquil['Latitude'], guayaquil['Longitude'], guayaquil['Site']):
    label = '{}'.format(Site)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_guayaquil)  
    
map_guayaquil

Let's create Quito's map.

In [ ]:
address1 = 'Quito, EC'

geolocator = Nominatim(user_agent="Quito")
location1 = geolocator.geocode(address1)
latitude_quito = location1.latitude
longitude_quito = location1.longitude
print('The geograpical coordinate of Quito are {}, {}.'.format(latitude_quito, longitude_quito))

The geograpical coordinate of Quito are -0.2201641, -78.5123274.


In [ ]:
map_quito = folium.Map(location=[latitude_quito, longitude_quito], zoom_start=10)

for lat, lng, Site in zip(quito['Latitude'], quito['Longitude'], quito['Site']):
    label = '{}'.format(Site)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quito)  
    
map_quito

Let's assing the Foursquare credentials into two variables in order to query the Foursquare database with the Api.

In [1]:
CLIENT_ID = 'YOUR_CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'YOUR_CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOUR_CLIENT_ID
CLIENT_SECRET:YOUR_CLIENT_SECRET


Let's assing the limit of venues that will return and the radius.

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


Create a function to get the venues per location.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Site', 
                  'Site Latitude', 
                  'Site Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Guayaquil Venues

In [ ]:
guayaquil_venues = getNearbyVenues(names=guayaquil['Site'],
                                   latitudes=guayaquil['Latitude'],
                                   longitudes=guayaquil['Longitude']
                                  )

Ayacucho
Bolivar (Sagrario)
Carbo (Concepcion)
Chongon.
Febres Cordero
Garcia Moreno
Guayaquil
Juan Gómez Rendón (Progreso)
Letamendi
Morro
Nueve De Octubre
Olmedo (San Alejo)
Pascuales
Playas (General Villamil)
Posorja
Puna
Roca
Rocafuerte
Sucre
Tarqui
Tenguel
Urdaneta
Ximena


In [ ]:
quito_venues = getNearbyVenues(names=quito['Site'],
                                   latitudes=quito['Latitude'],
                                   longitudes=quito['Longitude']
                                  )

Alangasi
Alfaro (Chimbacalle)
Amaguaña
Atahualpa (Chabaspamba)
Benalcazar
Calacali
Calderon (Carapungo)
Carcelen
Centro Historico
Chaupicruz (La Concepcion)
Chavezpamba
Checa (Chilpa)
Chilibulo
Chillogallo
Chimbacalle
Cochapamba
Comite Del Pueblo
Condado
Conocoto
Cotocollao
Cumbaya
El Batan
El Beaterio
El Inca
El Quinche
Gonzalez Suarez
Gualea
Guamani
Guangopolo
Guapulo
Guayllabamba
Iñaquito
Itchimbia
Jipijapa
Kennedy
La Concepción
La Ecuatoriana
La Floresta
La Libertad
La Magdalena
La Mena
La Merced
La Vicentina
Las Cuadras
Llano Chico
Lloa
Mariscal Sucre
Mindo
Nanegal
Nanegalito
Nayón
Nono
Pacto
Pedro Vicente Maldonado
Perucho
Pifo
Pintag
Pomasqui
Puellaro
Puembo
Puengasi
Puerto Quito
Quito
Quitumbe
Rumipamba
San Antonio
San Bartolo
San Blas
San Isidro Del Inca
San Jose De Minas
San Marcos
San Roque
San Sebastian
Santa Barbara
Santa Prisca
Tababela
Tumbaco
Villaflora
Yaruquì
Zàmbiza


In [ ]:
print(guayaquil_venues.shape)
guayaquil_venues.head()

(114, 7)


,Site,Site Latitude,Site Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ayacucho,-2.20548,-79.89029,Estadio George Capwell,-2.206698,-79.893780,Soccer Stadium
1,Ayacucho,-2.20548,-79.89029,Picanteria Cordero,-2.205780,-79.886116,Seafood Restaurant
2,Ayacucho,-2.20548,-79.89029,Ceviche D'Manolo,-2.207367,-79.888260,Seafood Restaurant
3,Ayacucho,-2.20548,-79.89029,Los Arbolitos,-2.207205,-79.888831,Seafood Restaurant
4,Ayacucho,-2.20548,-79.89029,Gran AKI Astillero,-2.207553,-79.886310,Supermarket


In [ ]:
guayaquil_venues['Site'].value_counts(), guayaquil_venues['Venue Category'].value_counts()

(Roca                         46
 Nueve De Octubre              8
 Urdaneta                      8
 Garcia Moreno                 7
 Ayacucho                      7
 Letamendi                     7
 Guayaquil                     6
 Bolivar (Sagrario)            6
 Pascuales                     4
 Tarqui                        4
 Morro                         3
 Carbo (Concepcion)            2
 Febres Cordero                2
 Puna                          1
 Chongon.                      1
 Posorja                       1
 Playas (General Villamil)     1
 Name: Site, dtype: int64, Seafood Restaurant           12
 Restaurant                    7
 Hotel                         5
 Sandwich Place                5
 Coffee Shop                   5
 Bar                           4
 Bakery                        4
 BBQ Joint                     4
 Juice Bar                     3
 Plaza                         3
 Pizza Place                   3
 Supermarket                   3
 Café            

In [ ]:
guayaquil_venues.groupby('Site').count()

,Site Latitude,Site Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Site,,,,,,
Ayacucho,7,7,7,7,7,7
Bolivar (Sagrario),6,6,6,6,6,6
Carbo (Concepcion),2,2,2,2,2,2
Chongon.,1,1,1,1,1,1
Febres Cordero,2,2,2,2,2,2
Garcia Moreno,7,7,7,7,7,7
Guayaquil,6,6,6,6,6,6
Letamendi,7,7,7,7,7,7
Morro,3,3,3,3,3,3


In [ ]:
print('There are {} uniques categories.'.format(len(guayaquil_venues['Venue Category'].unique())))

There are 56 uniques categories.


In [ ]:
# one hot encoding
guayaquil_onehot = pd.get_dummies(guayaquil_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
guayaquil_onehot['Site'] = guayaquil_venues['Site'] 

# move neighborhood column to the first column
fixed_columns = [guayaquil_onehot.columns[-1]] + list(guayaquil_onehot.columns[:-1])
guayaquil_onehot = guayaquil_onehot[fixed_columns]

guayaquil_onehot.head()

,Site,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Breakfast Spot,Building,Burger Joint,Café,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Food,Food Truck,Furniture / Home Store,Garden,Health Food Store,Hot Dog Joint,Hotel,Hotel Bar,Juice Bar,Latin American Restaurant,Lighthouse,Locksmith,Market,Movie Theater,Multiplex,Music Venue,Nightclub,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Port,Rest Area,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Theme Park
0,Ayacucho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Ayacucho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Ayacucho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Ayacucho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Ayacucho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
guayaquil_onehot.shape

(114, 57)

In [ ]:
guayaquil_grouped = guayaquil_onehot.groupby('Site').mean().reset_index()
guayaquil_grouped

,Site,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Breakfast Spot,Building,Burger Joint,Café,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Food,Food Truck,Furniture / Home Store,Garden,Health Food Store,Hot Dog Joint,Hotel,Hotel Bar,Juice Bar,Latin American Restaurant,Lighthouse,Locksmith,Market,Movie Theater,Multiplex,Music Venue,Nightclub,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Port,Rest Area,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Theme Park
0,Ayacucho,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.571429,0.000,0.00,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000
1,Bolivar (Sagrario),0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000,0.0,0.000000,0.000000,0.166667,0.166667,0.000,0.0,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Carbo (Concepcion),0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.5,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chongon.,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Febres Cordero,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.500000,0.000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Garcia Moreno,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.142857,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.571429,0.000000,0.142857,0.000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Guayaquil,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.333333,0.000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000
7,Letamendi,0.000000,0.000000,0.000000,0.0,0.00000

In [ ]:
guayaquil_grouped.shape

(17, 57)

In [ ]:
num_top_venues = 5

for hood in guayaquil_grouped['Site']:
    print("----"+hood+"----")
    temp = guayaquil_grouped[guayaquil_grouped['Site'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ayacucho----
                venue  freq
0  Seafood Restaurant  0.57
1         Supermarket  0.14
2           Juice Bar  0.14
3      Soccer Stadium  0.14
4           BBQ Joint  0.00


----Bolivar (Sagrario)----
                    venue  freq
0               Locksmith  0.17
1                  Market  0.17
2               Pet Store  0.17
3                Pharmacy  0.17
4  Furniture / Home Store  0.17


----Carbo (Concepcion)----
           venue  freq
0  Movie Theater   0.5
1      Multiplex   0.5
2      BBQ Joint   0.0
3         Bakery   0.0
4    Music Venue   0.0


----Chongon.----
           venue  freq
0     Playground   1.0
1      BBQ Joint   0.0
2         Bakery   0.0
3  Movie Theater   0.0
4      Multiplex   0.0


----Febres Cordero----
                venue  freq
0           BBQ Joint   0.5
1  Seafood Restaurant   0.5
2          Restaurant   0.0
3       Movie Theater   0.0
4           Multiplex   0.0


----Garcia Moreno----
                venue  freq
0      Sandwich Place  0.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Site']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Site'] = guayaquil_grouped['Site']

for ind in np.arange(guayaquil_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(guayaquil_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ayacucho,Seafood Restaurant,Supermarket,Soccer Stadium,Juice Bar,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden
1,Bolivar (Sagrario),Locksmith,Market,Furniture / Home Store,Pet Store,Pharmacy,Restaurant,Theme Park,Diner,Hot Dog Joint,Health Food Store
2,Carbo (Concepcion),Movie Theater,Multiplex,Theme Park,Diner,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store
3,Chongon.,Playground,Theme Park,Latin American Restaurant,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck
4,Febres Cordero,BBQ Joint,Seafood Restaurant,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck


### Guayaquil Cluster Analysis

In [ ]:
# set number of clusters
kclusters = 5

guayaquil_grouped_clustering = guayaquil_grouped.drop('Site', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(guayaquil_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 4, 0, 2, 1, 1, 1])

In [ ]:
guayaquil_grouped_clustering.head()

,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Breakfast Spot,Building,Burger Joint,Café,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Food,Food Truck,Furniture / Home Store,Garden,Health Food Store,Hot Dog Joint,Hotel,Hotel Bar,Juice Bar,Latin American Restaurant,Lighthouse,Locksmith,Market,Movie Theater,Multiplex,Music Venue,Nightclub,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Port,Rest Area,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Theme Park
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.571429,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.166667,0.166667,0.0,0.0,0.0,0.0,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.5,0.5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.500000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

guayaquil_merged = guayaquil

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
guayaquil_merged = guayaquil_merged.join(neighbourhoods_venues_sorted.set_index('Site'), on='Site')

guayaquil_merged.head() # check the last columns!

,PostalCode,Site,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,090101,Ayacucho,-2.20548,-79.89029,1.0,Seafood Restaurant,Supermarket,Soccer Stadium,Juice Bar,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden
1,090102,Bolivar (Sagrario),-2.19923,-79.88992,1.0,Locksmith,Market,Furniture / Home Store,Pet Store,Pharmacy,Restaurant,Theme Park,Diner,Hot Dog Joint,Health Food Store
2,090103,Carbo (Concepcion),-1.81207,-80.23183,1.0,Movie Theater,Multiplex,Theme Park,Diner,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store
3,090151,Chongon.,-2.23591,-80.07755,3.0,Playground,Theme Park,Latin American Restaurant,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck
4,090104,Febres Cordero,-2.20340,-79.93424,1.0,BBQ Joint,Seafood Restaurant,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck


In [ ]:
guayaquil_merged.dropna(inplace=True)

In [ ]:
guayaquil_merged['Cluster Labels']=guayaquil_merged['Cluster Labels'].astype('int64')

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude_guayaquil, longitude_guayaquil], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(guayaquil_merged['Latitude'], guayaquil_merged['Longitude'], guayaquil_merged['Site'], guayaquil_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Guayaquil Clusters

#### Guayaquil Cluster 1

In [ ]:
#Cluster 1
guayaquil_merged.loc[guayaquil_merged['Cluster Labels'] == 0, guayaquil_merged.columns[[1] + list(range(5, guayaquil_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Posorja,Port,Theme Park,Latin American Restaurant,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck


#### Guayaquil Cluster 2

In [ ]:
#Cluster 2
guayaquil_merged.loc[guayaquil_merged['Cluster Labels'] == 1, guayaquil_merged.columns[[1] + list(range(5, guayaquil_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ayacucho,Seafood Restaurant,Supermarket,Soccer Stadium,Juice Bar,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden
1,Bolivar (Sagrario),Locksmith,Market,Furniture / Home Store,Pet Store,Pharmacy,Restaurant,Theme Park,Diner,Hot Dog Joint,Health Food Store
2,Carbo (Concepcion),Movie Theater,Multiplex,Theme Park,Diner,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store
4,Febres Cordero,BBQ Joint,Seafood Restaurant,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck
5,Garcia Moreno,Sandwich Place,Breakfast Spot,Food,Seafood Restaurant,Theme Park,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store
6,Guayaquil,Seafood Restaurant,Restaurant,Stadium,Furniture / Home Store,Coffee Shop,Department Store,Hotel,Hot Dog Joint,Health Food Store,Garden
8,Letamendi,Restaurant,South Indian Restaurant,Food Truck,Pharmacy,Fast Food Restaurant,Convenience Store,Dessert Shop,Hotel,Hot Dog Joint,Health Food Store
9,Morro,Bakery,Sports Club,Rest Area,Theme Park,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden
10,Nueve De Octubre,Pizza Place,Department Store,Hot Dog Joint,Breakfast Spot,Snack Place,Coffee Shop,Diner,Hotel Bar,Hotel,Health Food Store
12,Pascuales,Bakery,Health Food Store,South American Restaurant,Soccer Field,Theme Park,Dessert Shop,Hotel Bar,Hotel,Hot Dog Joint,Garden


#### Guayaquil Cluster 3

In [ ]:
#Cluster 3
guayaquil_merged.loc[guayaquil_merged['Cluster Labels'] == 2, guayaquil_merged.columns[[1] + list(range(5, guayaquil_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Puna,Beach,Theme Park,Diner,Juice Bar,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store


#### Guayaquil Cluster 4

In [ ]:
#Cluster 4
guayaquil_merged.loc[guayaquil_merged['Cluster Labels'] == 3, guayaquil_merged.columns[[1] + list(range(5, guayaquil_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Chongon.,Playground,Theme Park,Latin American Restaurant,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck


#### Guayaquil Cluster 5

In [ ]:
#Cluster 5
guayaquil_merged.loc[guayaquil_merged['Cluster Labels'] == 4, guayaquil_merged.columns[[1] + list(range(5, guayaquil_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Playas (General Villamil),Supermarket,Theme Park,Juice Bar,Hotel Bar,Hotel,Hot Dog Joint,Health Food Store,Garden,Furniture / Home Store,Food Truck


Now, it has been created the same analysis for Quito.

### Quito Venues

In [ ]:
print(quito_venues.shape)
quito_venues.head()

(629, 7)


,Site,Site Latitude,Site Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alangasi,-0.30505,-78.41344,Parque De Alangasi,-0.307701,-78.415412,Park
1,Alfaro (Chimbacalle),-0.23333,-78.51667,Museo Interactivo de Ciencia,-0.236313,-78.516309,Science Museum
2,Alfaro (Chimbacalle),-0.23333,-78.51667,MIC,-0.231879,-78.515172,Science Museum
3,Alfaro (Chimbacalle),-0.23333,-78.51667,Mesón de la Recoleta,-0.231333,-78.512786,South American Restaurant
4,Atahualpa (Chabaspamba),-0.18439,-78.49171,Magic Sound & Music,-0.183566,-78.494332,Music Venue


In [ ]:
quito_venues['Site'].value_counts(), quito_venues['Venue Category'].value_counts()

(Benalcazar                    70
 Gonzalez Suarez               57
 Jipijapa                      52
 Cumbaya                       44
 La Floresta                   34
 San Roque                     31
 San Blas                      29
 Mariscal Sucre                27
 Guapulo                       25
 El Batan                      21
 Chaupicruz (La Concepcion)    17
 El Inca                       16
 San Sebastian                 14
 Quitumbe                      12
 Santa Prisca                  10
 Cochapamba                     9
 Atahualpa (Chabaspamba)        9
 Itchimbia                      8
 Rumipamba                      8
 Kennedy                        8
 San Marcos                     7
 Tumbaco                        6
 Mindo                          6
 Carcelen                       6
 Centro Historico               6
 Quito                          5
 El Quinche                     5
 Cotocollao                     4
 Las Cuadras                    4
 Chimbacalle  

In [ ]:
quito_venues['Site'].unique()

array(['Alangasi', 'Alfaro (Chimbacalle)', 'Atahualpa (Chabaspamba)',
       'Benalcazar', 'Calderon (Carapungo)', 'Carcelen',
       'Centro Historico', 'Chaupicruz (La Concepcion)', 'Chimbacalle',
       'Cochapamba', 'Comite Del Pueblo', 'Condado', 'Conocoto',
       'Cotocollao', 'Cumbaya', 'El Batan', 'El Beaterio', 'El Inca',
       'El Quinche', 'Gonzalez Suarez', 'Guamani', 'Guapulo',
       'Guayllabamba', 'Itchimbia', 'Jipijapa', 'Kennedy',
       'La Ecuatoriana', 'La Floresta', 'La Magdalena', 'La Mena',
       'La Merced', 'La Vicentina', 'Las Cuadras', 'Mariscal Sucre',
       'Mindo', 'Nanegalito', 'Nayón', 'Nono', 'Pifo', 'Pomasqui',
       'Puellaro', 'Puembo', 'Puengasi', 'Quito', 'Quitumbe', 'Rumipamba',
       'San Antonio', 'San Bartolo', 'San Blas', 'San Isidro Del Inca',
       'San Jose De Minas', 'San Marcos', 'San Roque', 'San Sebastian',
       'Santa Prisca', 'Tababela', 'Tumbaco', 'Villaflora', 'Yaruquì',
       'Zàmbiza'], dtype=object)

In [ ]:
quito_venues['Site'].value_counts(), quito_venues['Venue Category'].value_counts()

(Benalcazar                    70
 Gonzalez Suarez               57
 Jipijapa                      52
 Cumbaya                       44
 La Floresta                   34
 San Roque                     31
 San Blas                      29
 Mariscal Sucre                27
 Guapulo                       25
 El Batan                      21
 Chaupicruz (La Concepcion)    17
 El Inca                       16
 San Sebastian                 14
 Quitumbe                      12
 Santa Prisca                  10
 Cochapamba                     9
 Atahualpa (Chabaspamba)        9
 Itchimbia                      8
 Rumipamba                      8
 Kennedy                        8
 San Marcos                     7
 Tumbaco                        6
 Mindo                          6
 Carcelen                       6
 Centro Historico               6
 Quito                          5
 El Quinche                     5
 Cotocollao                     4
 Las Cuadras                    4
 Chimbacalle  

In [ ]:
quito_venues.groupby('Site').count()

,Site Latitude,Site Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Site,,,,,,
Alangasi,1,1,1,1,1,1
Alfaro (Chimbacalle),3,3,3,3,3,3
Atahualpa (Chabaspamba),9,9,9,9,9,9
Benalcazar,70,70,70,70,70,70
Calderon (Carapungo),3,3,3,3,3,3
Carcelen,6,6,6,6,6,6
Centro Historico,6,6,6,6,6,6
Chaupicruz (La Concepcion),17,17,17,17,17,17
Chimbacalle,4,4,4,4,4,4


In [ ]:
print('There are {} uniques categories.'.format(len(quito_venues['Venue Category'].unique())))

There are 146 uniques categories.


In [ ]:
# one hot encoding
quito_onehot = pd.get_dummies(quito_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
quito_onehot['Site'] = quito_venues['Site'] 

# move neighborhood column to the first column
fixed_columns = [quito_onehot.columns[-1]] + list(quito_onehot.columns[:-1])
quito_onehot = quito_onehot[fixed_columns]

quito_onehot.head()

,Site,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach Bar,Bed & Breakfast,Beer Store,Big Box Store,Bike Shop,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Locksmith,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Service,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool Hall,Pub,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Alangasi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alfaro (Chimbacalle),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alfaro (Chimbacalle),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alfaro (Chimbacalle),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Atahualpa (Chabaspamba),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
quito_onehot.shape

(629, 147)

In [ ]:
quito_grouped = quito_onehot.groupby('Site').mean().reset_index()
quito_grouped

,Site,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach Bar,Bed & Breakfast,Beer Store,Big Box Store,Bike Shop,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Locksmith,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Service,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool Hall,Pub,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Alangasi,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,Alfaro (Chimbacalle),0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [ ]:
quito_grouped.shape

(60, 147)

In [ ]:
num_top_venues = 5

for hood in quito_grouped['Site']:
    print("----"+hood+"----")
    temp = quito_grouped[quito_grouped['Site'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alangasi----
                  venue  freq
0                  Park   1.0
1   American Restaurant   0.0
2                Resort   0.0
3  Other Great Outdoors   0.0
4  Pakistani Restaurant   0.0


----Alfaro (Chimbacalle)----
                       venue  freq
0             Science Museum  0.67
1  South American Restaurant  0.33
2        American Restaurant  0.00
3                   Pharmacy  0.00
4       Pakistani Restaurant  0.00


----Atahualpa (Chabaspamba)----
                venue  freq
0               Hotel  0.22
1                 Bar  0.22
2      Cosmetics Shop  0.11
3  Mexican Restaurant  0.11
4          Food Court  0.11


----Benalcazar----
                venue  freq
0  Italian Restaurant  0.10
1               Hotel  0.07
2         Coffee Shop  0.07
3              Bakery  0.06
4      Ice Cream Shop  0.04


----Calderon (Carapungo)----
                           venue  freq
0                    Pizza Place  0.33
1                           Park  0.33
2             Chinese R

              venue  freq
0         Beach Bar   0.2
1         Locksmith   0.2
2  Video Game Store   0.2
3     Historic Site   0.2
4               Bar   0.2


----Quitumbe----
                 venue  freq
0          Pizza Place  0.33
1  Fried Chicken Joint  0.25
2           Restaurant  0.08
3        Big Box Store  0.08
4            BBQ Joint  0.08


----Rumipamba----
                venue  freq
0                 Spa  0.12
1  Mexican Restaurant  0.12
2         Music Venue  0.12
3          Food Court  0.12
4            Pharmacy  0.12


----San Antonio----
               venue  freq
0        Pizza Place  0.25
1         Restaurant  0.25
2     Ice Cream Shop  0.25
3  Food & Drink Shop  0.25
4           Pharmacy  0.00


----San Bartolo----
                           venue  freq
0               Department Store  0.67
1              Convenience Store  0.33
2                    Pizza Place  0.00
3  Paper / Office Supplies Store  0.00
4                           Park  0.00


----San Blas----
    

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Site']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted_quito = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted_quito['Site'] = quito_grouped['Site']

for ind in np.arange(quito_grouped.shape[0]):
    neighbourhoods_venues_sorted_quito.iloc[ind, 1:] = return_most_common_venues(quito_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted_quito.head()

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alangasi,Park,Wings Joint,Cosmetics Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
1,Alfaro (Chimbacalle),Science Museum,South American Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Wings Joint
2,Atahualpa (Chabaspamba),Hotel,Bar,Cosmetics Shop,Mexican Restaurant,Cajun / Creole Restaurant,Food Court,Music Venue,Historic Site,Food Truck,Department Store
3,Benalcazar,Italian Restaurant,Coffee Shop,Hotel,Bakery,Ice Cream Shop,Movie Theater,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Sandwich Place
4,Calderon (Carapungo),Pizza Place,Chinese Restaurant,Park,Wings Joint,Farmers Market,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant


### Quito Cluster Analysis

In [ ]:
# set number of clusters
kclusters_quito = 5

quito_grouped_clustering = quito_grouped.drop('Site', 1)

# run k-means clustering
kmeans_quito = KMeans(n_clusters=kclusters_quito, random_state=0).fit(quito_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_quito.labels_[0:100] 

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 4, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 4, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1])

In [ ]:
quito_grouped_clustering.head()

,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach Bar,Bed & Breakfast,Beer Store,Big Box Store,Bike Shop,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Locksmith,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Service,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool Hall,Pub,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.666667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.222222,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.111111,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted_quito.insert(0, 'Cluster Labels', kmeans_quito.labels_)

quito_merged = quito

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
quito_merged = quito_merged.join(neighbourhoods_venues_sorted_quito.set_index('Site'), on='Site')

quito_merged.head() # check the last columns!

,PostalCode,Site,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,170151,Alangasi,-0.30505,-78.41344,0.0,Park,Wings Joint,Cosmetics Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
1,170101,Alfaro (Chimbacalle),-0.23333,-78.51667,1.0,Science Museum,South American Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Wings Joint
2,170152,Amaguaña,-0.38084,-78.51544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,170153,Atahualpa (Chabaspamba),-0.18439,-78.49171,1.0,Hotel,Bar,Cosmetics Shop,Mexican Restaurant,Cajun / Creole Restaurant,Food Court,Music Venue,Historic Site,Food Truck,Department Store
4,170102,Benalcazar,-0.18262,-78.48122,1.0,Italian Restaurant,Coffee Shop,Hotel,Bakery,Ice Cream Shop,Movie Theater,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Sandwich Place


In [ ]:
quito_merged.dropna(inplace=True)

In [ ]:
quito_merged['Cluster Labels']=quito_merged['Cluster Labels'].astype('int64')

In [ ]:
# create map
map_clusters_quito = folium.Map(location=[latitude_quito, longitude_quito], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters_quito)
ys = [i + x + (i*x)**2 for i in range(kclusters_quito)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(quito_merged['Latitude'], quito_merged['Longitude'], quito_merged['Site'], quito_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_quito)
       
map_clusters_quito

#### Quito Cluster 1

In [ ]:
#Cluster 1
quito_merged.loc[quito_merged['Cluster Labels'] == 0, quito_merged.columns[[1] + list(range(5, quito_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alangasi,Park,Wings Joint,Cosmetics Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
22,El Beaterio,Park,Wings Joint,Cosmetics Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market
30,Guayllabamba,Restaurant,Park,Wings Joint,Farmers Market,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farm
42,La Vicentina,Park,Wings Joint,Cosmetics Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market


#### Quito Cluster 2

In [ ]:
#Cluster 2
quito_merged.loc[quito_merged['Cluster Labels'] == 1, quito_merged.columns[[1] + list(range(5, quito_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Alfaro (Chimbacalle),Science Museum,South American Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Wings Joint
3,Atahualpa (Chabaspamba),Hotel,Bar,Cosmetics Shop,Mexican Restaurant,Cajun / Creole Restaurant,Food Court,Music Venue,Historic Site,Food Truck,Department Store
4,Benalcazar,Italian Restaurant,Coffee Shop,Hotel,Bakery,Ice Cream Shop,Movie Theater,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Sandwich Place
6,Calderon (Carapungo),Pizza Place,Chinese Restaurant,Park,Wings Joint,Farmers Market,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant
7,Carcelen,Farmers Market,Soccer Field,Pizza Place,Seafood Restaurant,BBQ Joint,Gym,Wings Joint,Food & Drink Shop,Food,Flea Market
8,Centro Historico,Breakfast Spot,Burger Joint,Pizza Place,Restaurant,Seafood Restaurant,Stadium,Farmers Market,Food & Drink Shop,Food,Flea Market
9,Chaupicruz (La Concepcion),Pizza Place,Restaurant,Seafood Restaurant,Sculpture Garden,Fried Chicken Joint,Pakistani Restaurant,Mexican Restaurant,Bar,Chinese Restaurant,Asian Restaurant
14,Chimbacalle,Bakery,Diner,Bus Station,Stadium,Wings Joint,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market
15,Cochapamba,Furniture / Home Store,Record Shop,Ice Cream Shop,Department Store,Toy / Game Store,Sandwich Place,Supermarket,Event Space,Food,Flea Market
16,Comite Del Pueblo,Wings Joint,Latin American Restaurant,Farmers Market,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farm


#### Quito Cluster 3

In [ ]:
#Cluster 3
quito_merged.loc[quito_merged['Cluster Labels'] == 2, quito_merged.columns[[1] + list(range(5, quito_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Nono,Recreation Center,Cosmetics Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm
59,Puembo,Recreation Center,Athletics & Sports,Farm,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market


#### Quito Cluster 4

In [ ]:
#Cluster 4
quito_merged.loc[quito_merged['Cluster Labels'] == 3, quito_merged.columns[[1] + list(range(5, quito_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,Yaruquì,Cafeteria,Farm,Wings Joint,Fast Food Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Farmers Market


#### Quito Cluster 5

In [ ]:
#Cluster 5
quito_merged.loc[quito_merged['Cluster Labels'] == 4, quito_merged.columns[[1] + list(range(5, quito_merged.shape[1]))]]

,Site,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,La Ecuatoriana,Fried Chicken Joint,Seafood Restaurant,French Restaurant,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm
55,Pifo,Seafood Restaurant,Wings Joint,French Restaurant,Food Court,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm
